# 📊 AltScore — Data Integration & Merging


In [1]:
# Core libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8-whitegrid')

# Settings
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

print('✅ Libraries loaded successfully!')
print(f'Pandas: {pd.__version__}, NumPy: {np.__version__}')

✅ Libraries loaded successfully!
Pandas: 2.3.3, NumPy: 2.4.0


---
## STEP 1 — Load Main Application Data

In [ ]:

DATA_DIR = '../data/raw/'  # Change this to where your CSV files are stored

print('📂 Loading main application data...')
app_train = pd.read_csv(DATA_DIR + 'application_train.csv')

print(f'✅ Loaded: {app_train.shape[0]:,} applications × {app_train.shape[1]} features')
print(f'Target (TARGET): {app_train["TARGET"].value_counts().to_dict()}')
print(f'Default rate: {app_train["TARGET"].mean()*100:.2f}%')

# Keep a copy of the target variable
y_train = app_train['TARGET'].copy()

print('\n=== First 3 rows ===')
display(app_train.head(3))

---
##  STEP 2 — Load & Aggregate Bureau Data


In [ ]:
print('📂 Loading bureau data...')
bureau = pd.read_csv(DATA_DIR + 'bureau.csv')

print(f'✅ Loaded: {bureau.shape[0]:,} bureau records')
print(f'Unique customers: {bureau["SK_ID_CURR"].nunique():,}')
print(f'Coverage: {bureau["SK_ID_CURR"].nunique() / app_train.shape[0] * 100:.1f}% of applications')

display(bureau.head(3))

In [ ]:
# ─── Aggregate Bureau Data ─────────────────────────────────────────────────────
print('🔧 Aggregating bureau data by customer...')

# Count of bureau records per customer
bureau_agg = bureau.groupby('SK_ID_CURR').agg({
    'SK_ID_BUREAU': 'count',  # Number of credit lines
}).rename(columns={'SK_ID_BUREAU': 'BUREAU_SK_ID_BUREAU_count'})

# Aggregate numeric columns
numeric_cols = bureau.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols.remove('SK_ID_CURR')  # Don't aggregate the ID itself
numeric_cols.remove('SK_ID_BUREAU')

for col in numeric_cols:
    # Sum, mean, max, min for credit amounts and days
    if 'AMT' in col or 'CNT' in col:
        bureau_agg[f'BUREAU_{col}_sum'] = bureau.groupby('SK_ID_CURR')[col].sum()
        bureau_agg[f'BUREAU_{col}_mean'] = bureau.groupby('SK_ID_CURR')[col].mean()
        bureau_agg[f'BUREAU_{col}_max'] = bureau.groupby('SK_ID_CURR')[col].max()
        if 'AMT' in col:
            bureau_agg[f'BUREAU_{col}_min'] = bureau.groupby('SK_ID_CURR')[col].min()
    elif 'DAYS' in col:
        bureau_agg[f'BUREAU_{col}_min'] = bureau.groupby('SK_ID_CURR')[col].min()
        bureau_agg[f'BUREAU_{col}_max'] = bureau.groupby('SK_ID_CURR')[col].max()
        bureau_agg[f'BUREAU_{col}_mean'] = bureau.groupby('SK_ID_CURR')[col].mean()

# Count active credits
if 'CREDIT_ACTIVE' in bureau.columns:
    bureau_agg['BUREAU_ACTIVE_COUNT'] = bureau[bureau['CREDIT_ACTIVE'] == 'Active'].groupby('SK_ID_CURR').size()

# Count different credit types
if 'CREDIT_CURRENCY' in bureau.columns:
    bureau_agg['BUREAU_CURRENCY_COUNT'] = bureau.groupby('SK_ID_CURR')['CREDIT_CURRENCY'].nunique()
if 'CREDIT_TYPE' in bureau.columns:
    bureau_agg['BUREAU_TYPE_COUNT'] = bureau.groupby('SK_ID_CURR')['CREDIT_TYPE'].nunique()

bureau_agg = bureau_agg.reset_index()

print(f'✅ Created {bureau_agg.shape[1]-1} bureau aggregated features')
print(f'Customers with bureau data: {bureau_agg.shape[0]:,}')

display(bureau_agg.head(3))

### 2.1 — Bureau Balance (Monthly Credit Behavior)

In [ ]:
print('📂 Loading bureau_balance data...')
bureau_balance = pd.read_csv(DATA_DIR + 'bureau_balance.csv')

print(f'✅ Loaded: {bureau_balance.shape[0]:,} monthly balance records')
display(bureau_balance.head(3))

In [ ]:
# ─── Aggregate Bureau Balance ──────────────────────────────────────────────────
print('🔧 Aggregating bureau_balance...')

bb_agg = bureau_balance.groupby('SK_ID_BUREAU').agg({
    'MONTHS_BALANCE': ['min', 'max'],  # Range of months
}).reset_index()
bb_agg.columns = ['SK_ID_BUREAU', 'BB_MONTHS_MIN_min', 'BB_MONTHS_MAX_max']

# Count records per bureau ID
bb_counts = bureau_balance.groupby('SK_ID_BUREAU').size().reset_index(name='BB_MONTHS_COUNT_sum')
bb_agg = bb_agg.merge(bb_counts, on='SK_ID_BUREAU', how='left')
bb_agg['BB_MONTHS_COUNT_mean'] = bb_agg['BB_MONTHS_COUNT_sum'] / bb_agg['BB_MONTHS_COUNT_sum']

# Status analysis (if STATUS column exists)
if 'STATUS' in bureau_balance.columns:
    status_closed = bureau_balance[bureau_balance['STATUS'] == 'C'].groupby('SK_ID_BUREAU').size().reset_index(name='BB_STATUS_CLOSED_sum')
    bb_agg = bb_agg.merge(status_closed, on='SK_ID_BUREAU', how='left')
    bb_agg['BB_STATUS_CLOSED_mean'] = bb_agg['BB_STATUS_CLOSED_sum'] / bb_agg['BB_MONTHS_COUNT_sum']
    bb_agg['BB_STATUS_CLOSED_sum'].fillna(0, inplace=True)
    bb_agg['BB_STATUS_CLOSED_mean'].fillna(0, inplace=True)

# Merge back to bureau data
bureau_with_balance = bureau[['SK_ID_CURR', 'SK_ID_BUREAU']].merge(bb_agg, on='SK_ID_BUREAU', how='left')
bb_final = bureau_with_balance.groupby('SK_ID_CURR').mean().reset_index()

print(f'✅ Created {bb_final.shape[1]-1} bureau_balance features')

# Merge bureau + bureau_balance
bureau_full = bureau_agg.merge(bb_final, on='SK_ID_CURR', how='left')
print(f'✅ Combined bureau features: {bureau_full.shape[1]-1} total')

---
## STEP 3 — Load & Aggregate Previous Applications

In [ ]:
print('📂 Loading previous_application data...')
prev_app = pd.read_csv(DATA_DIR + 'previous_application.csv')

print(f'✅ Loaded: {prev_app.shape[0]:,} previous applications')
print(f'Unique customers: {prev_app["SK_ID_CURR"].nunique():,}')
print(f'Coverage: {prev_app["SK_ID_CURR"].nunique() / app_train.shape[0] * 100:.1f}% of applications')

display(prev_app.head(3))

In [ ]:
# ─── Aggregate Previous Applications ───────────────────────────────────────────
print('🔧 Aggregating previous_application...')

# Count of previous applications
prev_agg = prev_app.groupby('SK_ID_CURR').agg({
    'SK_ID_PREV': 'count'
}).rename(columns={'SK_ID_PREV': 'PREV_SK_ID_PREV_count'})

# Aggregate numeric columns
numeric_cols = prev_app.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols = [c for c in numeric_cols if c not in ['SK_ID_CURR', 'SK_ID_PREV']]

for col in numeric_cols:
    if 'AMT' in col:
        prev_agg[f'PREV_{col}_mean'] = prev_app.groupby('SK_ID_CURR')[col].mean()
        prev_agg[f'PREV_{col}_max'] = prev_app.groupby('SK_ID_CURR')[col].max()
        prev_agg[f'PREV_{col}_min'] = prev_app.groupby('SK_ID_CURR')[col].min()
    elif 'DAYS' in col:
        prev_agg[f'PREV_{col}_mean'] = prev_app.groupby('SK_ID_CURR')[col].mean()
        prev_agg[f'PREV_{col}_max'] = prev_app.groupby('SK_ID_CURR')[col].max()
        prev_agg[f'PREV_{col}_min'] = prev_app.groupby('SK_ID_CURR')[col].min()
    elif 'RATE' in col or 'HOUR' in col:
        prev_agg[f'PREV_{col}_mean'] = prev_app.groupby('SK_ID_CURR')[col].mean()
        prev_agg[f'PREV_{col}_max'] = prev_app.groupby('SK_ID_CURR')[col].max()
        prev_agg[f'PREV_{col}_min'] = prev_app.groupby('SK_ID_CURR')[col].min()
    elif 'NFLAG' in col or 'FLAG' in col:
        prev_agg[f'PREV_{col}_sum'] = prev_app.groupby('SK_ID_CURR')[col].sum()
        prev_agg[f'PREV_{col}_mean'] = prev_app.groupby('SK_ID_CURR')[col].mean()

# Categorical aggregations
if 'NAME_CONTRACT_STATUS' in prev_app.columns:
    prev_agg['PREV_APPROVED_COUNT'] = prev_app[prev_app['NAME_CONTRACT_STATUS'] == 'Approved'].groupby('SK_ID_CURR').size()
if 'NAME_CONTRACT_TYPE' in prev_app.columns:
    prev_agg['PREV_CONSUMER_LOAN_COUNT'] = prev_app[prev_app['NAME_CONTRACT_TYPE'] == 'Consumer loans'].groupby('SK_ID_CURR').size()
if 'NAME_PAYMENT_TYPE' in prev_app.columns:
    prev_agg['PREV_CASH_PAYMENT_COUNT'] = prev_app[prev_app['NAME_PAYMENT_TYPE'] == 'Cash through the bank'].groupby('SK_ID_CURR').size()
if 'NAME_CLIENT_TYPE' in prev_app.columns:
    prev_agg['PREV_REPEATER_COUNT'] = prev_app[prev_app['NAME_CLIENT_TYPE'] == 'Repeater'].groupby('SK_ID_CURR').size()

# Diversity metrics (number of unique categories)
for col in ['NAME_GOODS_CATEGORY', 'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE', 'CHANNEL_TYPE', 'NAME_SELLER_INDUSTRY', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION']:
    if col in prev_app.columns:
        prev_agg[f'PREV_{col}_NUNIQUE'] = prev_app.groupby('SK_ID_CURR')[col].nunique()

prev_agg = prev_agg.reset_index()
prev_agg.fillna(0, inplace=True)

print(f'✅ Created {prev_agg.shape[1]-1} previous_application features')
print(f'Customers with previous apps: {prev_agg.shape[0]:,}')

display(prev_agg.head(3))

---
##  STEP 4 — Load & Aggregate POS Cash Balance

In [ ]:
print('📂 Loading POS_CASH_balance data...')
pos_cash = pd.read_csv(DATA_DIR + 'POS_CASH_balance.csv')

print(f'✅ Loaded: {pos_cash.shape[0]:,} POS/cash balance records')
print(f'Unique customers: {pos_cash["SK_ID_CURR"].nunique():,}')

display(pos_cash.head(3))

In [ ]:
# ─── Aggregate POS Cash Balance ────────────────────────────────────────────────
print('🔧 Aggregating POS_CASH_balance...')

# First aggregate by SK_ID_PREV (loan level)
pos_agg = pos_cash.groupby('SK_ID_PREV').agg({
    'MONTHS_BALANCE': ['min', 'max', 'mean', 'size'],
    'CNT_INSTALMENT': ['mean', 'max', 'sum'],
    'CNT_INSTALMENT_FUTURE': ['mean', 'max', 'sum'],
    'SK_DPD': ['mean', 'max', 'sum'],
    'SK_DPD_DEF': ['mean', 'max', 'sum'],
}).reset_index()

pos_agg.columns = ['SK_ID_PREV', 
                   'POS_MONTHS_BALANCE_min', 'POS_MONTHS_BALANCE_max', 'POS_MONTHS_BALANCE_mean', 'POS_MONTHS_BALANCE_size',
                   'POS_CNT_INSTALMENT_mean', 'POS_CNT_INSTALMENT_max', 'POS_CNT_INSTALMENT_sum',
                   'POS_CNT_INSTALMENT_FUTURE_mean', 'POS_CNT_INSTALMENT_FUTURE_max', 'POS_CNT_INSTALMENT_FUTURE_sum',
                   'POS_SK_DPD_mean', 'POS_SK_DPD_max', 'POS_SK_DPD_sum',
                   'POS_SK_DPD_DEF_mean', 'POS_SK_DPD_DEF_max', 'POS_SK_DPD_DEF_sum']

# Count active loans (NAME_CONTRACT_STATUS = 'Active')
if 'NAME_CONTRACT_STATUS' in pos_cash.columns:
    active_count = pos_cash[pos_cash['NAME_CONTRACT_STATUS'] == 'Active'].groupby('SK_ID_PREV')['SK_ID_PREV'].count().reset_index(name='POS_ACTIVE_COUNT')
    pos_agg = pos_agg.merge(active_count, on='SK_ID_PREV', how='left')
    pos_agg['POS_ACTIVE_COUNT'].fillna(0, inplace=True)

# Payment regularity score
pos_agg['POS_PAYMENT_REGULARITY'] = 1.0 - (pos_agg['POS_SK_DPD_sum'] / (pos_agg['POS_MONTHS_BALANCE_size'] + 1e-5))

# Get SK_ID_CURR mapping
pos_curr_map = pos_cash[['SK_ID_CURR', 'SK_ID_PREV']].drop_duplicates()
pos_agg = pos_curr_map.merge(pos_agg, on='SK_ID_PREV', how='left')

# Aggregate to customer level
pos_final = pos_agg.groupby('SK_ID_CURR').agg({
    'SK_ID_PREV': 'count',
    'POS_MONTHS_BALANCE_min': 'min',
    'POS_MONTHS_BALANCE_max': 'max',
    'POS_MONTHS_BALANCE_mean': 'mean',
    'POS_MONTHS_BALANCE_size': 'mean',
    'POS_CNT_INSTALMENT_mean': 'mean',
    'POS_CNT_INSTALMENT_max': 'max',
    'POS_CNT_INSTALMENT_sum': 'sum',
    'POS_CNT_INSTALMENT_FUTURE_mean': 'mean',
    'POS_CNT_INSTALMENT_FUTURE_max': 'max',
    'POS_CNT_INSTALMENT_FUTURE_sum': 'sum',
    'POS_SK_DPD_mean': 'mean',
    'POS_SK_DPD_max': 'max',
    'POS_SK_DPD_sum': 'sum',
    'POS_SK_DPD_DEF_mean': 'mean',
    'POS_SK_DPD_DEF_max': 'max',
    'POS_SK_DPD_DEF_sum': 'sum',
    'POS_ACTIVE_COUNT': 'sum',
    'POS_PAYMENT_REGULARITY': 'mean',
}).reset_index()

pos_final.rename(columns={'SK_ID_PREV': 'POS_SK_ID_PREV_count'}, inplace=True)

print(f'✅ Created {pos_final.shape[1]-1} POS_CASH features')
print(f'Customers with POS data: {pos_final.shape[0]:,}')

display(pos_final.head(3))

---
## STEP 5 — Load & Aggregate Credit Card Balance

In [ ]:
print('📂 Loading credit_card_balance data...')
cc_bal = pd.read_csv(DATA_DIR + 'credit_card_balance.csv')

print(f'✅ Loaded: {cc_bal.shape[0]:,} credit card balance records')
print(f'Unique customers: {cc_bal["SK_ID_CURR"].nunique():,}')

display(cc_bal.head(3))

In [ ]:
# ─── Aggregate Credit Card Balance ─────────────────────────────────────────────
print('🔧 Aggregating credit_card_balance...')

# Aggregate by SK_ID_PREV first
cc_prev_agg = cc_bal.groupby('SK_ID_PREV').agg({
    'MONTHS_BALANCE': ['min', 'max', 'mean', 'size'],
    'AMT_BALANCE': ['mean', 'max', 'min', 'sum'],
    'AMT_CREDIT_LIMIT_ACTUAL': ['mean', 'max', 'min'],
    'AMT_DRAWINGS_ATM_CURRENT': 'sum',
    'AMT_DRAWINGS_CURRENT': ['mean', 'max', 'sum'],
    'AMT_DRAWINGS_OTHER_CURRENT': 'sum',
    'AMT_DRAWINGS_POS_CURRENT': 'sum',
    'AMT_INST_MIN_REGULARITY': ['mean', 'max', 'min'],
    'AMT_PAYMENT_CURRENT': 'sum',
    'AMT_PAYMENT_TOTAL_CURRENT': ['mean', 'max', 'min', 'sum'],
    'AMT_RECEIVABLE_PRINCIPAL': ['mean', 'max', 'min', 'sum'],
    'AMT_RECIVABLE': ['mean', 'max', 'min', 'sum'],
    'AMT_TOTAL_RECEIVABLE': ['mean', 'max', 'min', 'sum'],
    'CNT_DRAWINGS_ATM_CURRENT': 'sum',
    'CNT_DRAWINGS_CURRENT': ['mean', 'max', 'sum'],
    'CNT_DRAWINGS_OTHER_CURRENT': 'sum',
    'CNT_DRAWINGS_POS_CURRENT': 'sum',
    'CNT_INSTALMENT_MATURE_CUM': ['mean', 'max'],
    'SK_DPD': ['mean', 'max', 'sum'],
    'SK_DPD_DEF': ['mean', 'max', 'sum'],
}).reset_index()

cc_prev_agg.columns = ['_'.join(col).strip('_') if col[1] else col[0] for col in cc_prev_agg.columns.values]
cc_prev_agg.columns = ['SK_ID_PREV'] + ['CC_' + c for c in cc_prev_agg.columns[1:]]

# Credit utilization
cc_prev_agg['CC_UTILIZATION_MEAN'] = cc_prev_agg['CC_AMT_BALANCE_mean'] / (cc_prev_agg['CC_AMT_CREDIT_LIMIT_ACTUAL_mean'] + 1e-5)
cc_prev_agg['CC_UTILIZATION_MAX'] = cc_prev_agg['CC_AMT_BALANCE_max'] / (cc_prev_agg['CC_AMT_CREDIT_LIMIT_ACTUAL_max'] + 1e-5)
cc_prev_agg['CC_UTILIZATION_MIN'] = cc_prev_agg['CC_AMT_BALANCE_min'] / (cc_prev_agg['CC_AMT_CREDIT_LIMIT_ACTUAL_min'] + 1e-5)

# Map to SK_ID_CURR
cc_curr_map = cc_bal[['SK_ID_CURR', 'SK_ID_PREV']].drop_duplicates()
cc_prev_agg = cc_curr_map.merge(cc_prev_agg, on='SK_ID_PREV', how='left')

# Aggregate to customer level
cc_final = cc_prev_agg.groupby('SK_ID_CURR').mean().reset_index()
cc_final.rename(columns={'SK_ID_PREV': 'CC_SK_ID_PREV_count'}, inplace=True)

print(f'✅ Created {cc_final.shape[1]-1} credit_card features')
print(f'Customers with CC data: {cc_final.shape[0]:,}')

display(cc_final.head(3))

---
## STEP 6 — Load & Aggregate Installments Payments

In [ ]:
print('📂 Loading installments_payments data...')
inst_pay = pd.read_csv(DATA_DIR + 'installments_payments.csv')

print(f'✅ Loaded: {inst_pay.shape[0]:,} installment payment records')
print(f'Unique customers: {inst_pay["SK_ID_CURR"].nunique():,}')

display(inst_pay.head(3))

In [ ]:
# ─── Aggregate Installments Payments ───────────────────────────────────────────
print('🔧 Aggregating installments_payments...')

# Payment difference (paid - expected)
inst_pay['PAYMENT_DIFF'] = inst_pay['AMT_PAYMENT'] - inst_pay['AMT_INSTALMENT']
inst_pay['PAYMENT_RATIO'] = inst_pay['AMT_PAYMENT'] / (inst_pay['AMT_INSTALMENT'] + 1e-5)

# Aggregate by SK_ID_PREV
inst_prev_agg = inst_pay.groupby('SK_ID_PREV').agg({
    'NUM_INSTALMENT_VERSION': ['mean', 'max', 'nunique'],
    'NUM_INSTALMENT_NUMBER': ['mean', 'max'],
    'DAYS_INSTALMENT': ['mean', 'max', 'min'],
    'DAYS_ENTRY_PAYMENT': ['mean', 'max', 'min'],
    'AMT_INSTALMENT': ['mean', 'max', 'min', 'sum'],
    'AMT_PAYMENT': ['mean', 'max', 'min', 'sum'],
    'PAYMENT_DIFF': ['mean', 'max', 'min'],
    'PAYMENT_RATIO': ['mean', 'max', 'min'],
}).reset_index()

inst_prev_agg.columns = ['_'.join(col).strip('_') if col[1] else col[0] for col in inst_prev_agg.columns.values]
inst_prev_agg.columns = ['SK_ID_PREV'] + ['INST_' + c for c in inst_prev_agg.columns[1:]]

# Payment behavior flags
# Early payment: paid before due date
early_payments = inst_pay[inst_pay['DAYS_ENTRY_PAYMENT'] < inst_pay['DAYS_INSTALMENT']].groupby('SK_ID_PREV').size()
# Late payment: paid after due date
late_payments = inst_pay[inst_pay['DAYS_ENTRY_PAYMENT'] > inst_pay['DAYS_INSTALMENT']].groupby('SK_ID_PREV').size()
# Full payment: paid >= expected
full_payments = inst_pay[inst_pay['AMT_PAYMENT'] >= inst_pay['AMT_INSTALMENT']].groupby('SK_ID_PREV').size()
# Total payments
total_payments = inst_pay.groupby('SK_ID_PREV').size()

inst_prev_agg['INST_EARLY_PAYMENT_RATE'] = (early_payments / total_payments).fillna(0)
inst_prev_agg['INST_LATE_PAYMENT_RATE'] = (late_payments / total_payments).fillna(0)
inst_prev_agg['INST_FULL_PAYMENT_RATE'] = (full_payments / total_payments).fillna(0)

# Map to SK_ID_CURR
inst_curr_map = inst_pay[['SK_ID_CURR', 'SK_ID_PREV']].drop_duplicates()
inst_prev_agg = inst_curr_map.merge(inst_prev_agg, on='SK_ID_PREV', how='left')

# Aggregate to customer level
inst_final = inst_prev_agg.groupby('SK_ID_CURR').mean().reset_index()
inst_final.rename(columns={'SK_ID_PREV': 'INST_SK_ID_PREV_count'}, inplace=True)

print(f'✅ Created {inst_final.shape[1]-1} installments_payments features')
print(f'Customers with installment data: {inst_final.shape[0]:,}')

display(inst_final.head(3))

---
## STEP 7 — Merge All Datasets

Now we merge all aggregated features back to the main application data.

In [ ]:
print('🔧 Merging all datasets...')
print(f'Starting with: {app_train.shape}')

# Merge bureau
df = app_train.merge(bureau_full, on='SK_ID_CURR', how='left')
print(f'After bureau: {df.shape}')

# Merge previous applications
df = df.merge(prev_agg, on='SK_ID_CURR', how='left')
print(f'After prev_app: {df.shape}')

# Merge POS cash
df = df.merge(pos_final, on='SK_ID_CURR', how='left')
print(f'After POS_CASH: {df.shape}')

# Merge credit card
df = df.merge(cc_final, on='SK_ID_CURR', how='left')
print(f'After credit_card: {df.shape}')

# Merge installments
df = df.merge(inst_final, on='SK_ID_CURR', how='left')
print(f'After installments: {df.shape}')

print(f'\n✅ Final merged dataset: {df.shape}')
print(f'Total features: {df.shape[1]}')
print(f'Total rows: {df.shape[0]:,}')

---
## STEP 8 — Data Quality Check

In [ ]:
print('🔍 Checking merged data quality...')

# Missing values
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'feature': missing.index,
    'missing_count': missing.values,
    'missing_pct': missing_pct.values
}).sort_values('missing_pct', ascending=False)

high_missing = missing_df[missing_df.missing_pct > 50]
print(f'\nFeatures with >50% missing: {len(high_missing)}')

if len(high_missing) > 0:
    print('\nTop 10 features with most missing:')
    display(high_missing.head(10))

# Data types
print(f'\nData types:')
print(df.dtypes.value_counts())

In [ ]:
# Verify target distribution
print('🎯 Target distribution check:')
print(df['TARGET'].value_counts())
print(f'Default rate: {df["TARGET"].mean()*100:.2f}%')

# Verify no duplicate rows
print(f'\nDuplicate rows: {df.duplicated().sum()}')

# Memory usage
print(f'\nMemory usage: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB')

---
##  STEP 9 — Save Merged Dataset

In [ ]:
import os

# Create output directory if it doesn't exist
os.makedirs('../data/merged', exist_ok=True)

# Separate X and y
X_merged = df.drop(columns=['TARGET'])
y_merged = df['TARGET']

# Save to CSV
print('💾 Saving merged datasets...')
X_merged.to_csv('../data/merged/x_train_merged.csv', index=False)
y_merged.to_csv('../data/merged/y_train.csv', index=False, header=True)

print(f'\n✅ Saved:')
print(f'   x_train_merged.csv: {X_merged.shape}')
print(f'   y_train.csv: {y_merged.shape}')

# Also save full feature list
feature_list = pd.DataFrame({'feature': X_merged.columns})
feature_list.to_csv('../data/merged/feature_names.txt', index=False, header=False)
print(f'   feature_names.txt: {len(X_merged.columns)} features')

print('\n🎉 Data integration complete!')
print(f'\nYou now have a merged dataset with {X_merged.shape[1]} features ready for preprocessing.')

---
##  STEP 10 — Quick Summary Statistics

In [ ]:
print('📊 FINAL SUMMARY')
print('='*70)
print(f'Total applications:        {X_merged.shape[0]:,}')
print(f'Total features:            {X_merged.shape[1]:,}')
print(f'Default rate:              {y_merged.mean()*100:.2f}%')
print(f'Features from each source:')
print(f'  - Application (base):    ~122')
print(f'  - Bureau:                {len([c for c in X_merged.columns if c.startswith("BUREAU")])}')
print(f'  - Previous apps:         {len([c for c in X_merged.columns if c.startswith("PREV")])}')
print(f'  - POS/Cash:              {len([c for c in X_merged.columns if c.startswith("POS")])}')
print(f'  - Credit card:           {len([c for c in X_merged.columns if c.startswith("CC")])}')
print(f'  - Installments:          {len([c for c in X_merged.columns if c.startswith("INST")])}')
print('='*70)

# Create a summary report
summary = {
    'Total Applications': X_merged.shape[0],
    'Total Features': X_merged.shape[1],
    'Default Rate (%)': round(y_merged.mean()*100, 2),
    'Missing Value %': round(X_merged.isnull().sum().sum() / (X_merged.shape[0] * X_merged.shape[1]) * 100, 2),
}

summary_df = pd.DataFrame([summary])
summary_df.to_csv('../reports/data_merge_summary.csv', index=False)
print('\n✅ Summary saved to reports/data_merge_summary.csv')